<a href="https://colab.research.google.com/github/deviwlndr/714220054_Devi-Wulandari/blob/main/analisis_sentimen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Sastrawi
!pip install googletrans==4.0.0-rc1 langdetect



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.0 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=fbaaded11089626ed0cfa94b0f684fee399545ef98aff7561aa6dc8c5cbb35b8
  Stored in directory: /root/.cache/pip/wheels/

In [2]:
import pandas as pd
import numpy as np
import re
import string
import nltk

from langdetect import detect
from googletrans import Translator

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from wordcloud import WordCloud
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Analisis Sentimen/dataset_ulasan_museum_sri_baduga.csv')
df.head()


,title,url,stars,name,reviewUrl,text
0,Museum Sri Baduga,https://www.google.com/maps/search/?api=1&quer...,4,NaN,NaN,NaN
1,Museum Sri Baduga,https://www.google.com/maps/search/?api=1&quer...,5,NaN,NaN,Musieum Sribaduga
2,Museum Sri Baduga,https://www.google.com/maps/search/?api=1&quer...,5,NaN,NaN,NaN
3,Museum Sri Baduga,https://www.google.com/maps/search/?api=1&quer...,5,NaN,NaN,Museum di dalam kota bandung
4,Museum Sri Baduga,https://www.google.com/maps/search/?api=1&quer...,4,NaN,NaN,هذا المتحف هو رحلة عبر الزمن، حيث نرى بعض من ت...


In [5]:
df = df[['text', 'stars']]
df = df.dropna(subset=['text'])
df['stars'] = df['stars'].astype(int)

df.head()


,text,stars
1,Musieum Sribaduga,5
3,Museum di dalam kota bandung,5
4,هذا المتحف هو رحلة عبر الزمن، حيث نرى بعض من ت...,4
7,"Kesini pas study tour SMP, mungkin tahun 2018/...",5
8,Bagus sekali,5


In [6]:
def case_folding(text):
    return text.lower()

In [7]:
df['case_folding'] = df['text'].apply(case_folding)

In [8]:
# 3. CHARACTER FILTERING
# Menghapus angka, emoji, tanda baca, URL
# ---------------------------------------------
def clean_characters(text):
    text = re.sub(r'http\S+|www\S+|https\S+', ' ', text)   # hapus URL
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)              # hapus angka, emoji, simbol
    text = re.sub(r'\s+', ' ', text).strip()              # rapikan spasi
    return text

In [9]:
df['clean_text'] = df['case_folding'].apply(clean_characters)

In [10]:
translator = Translator()

def translate_to_indonesian(text):
    if not isinstance(text, str) or text.strip() == "":
        return text

    try:
        lang = detect(text)
    except:
        return text

    if lang == "en":  # hanya terjemahkan jika English
        try:
            return translator.translate(text, src='en', dest='id').text
        except:
            return text

    return text



In [11]:
df['translated'] = df['clean_text'].apply(translate_to_indonesian)
print(df[['clean_text','translated']].head())


                                clean_text  \
1                        musieum sribaduga   
3             museum di dalam kota bandung   
4                                            
7  kesini pas study tour smp mungkin tahun   
8                             bagus sekali   

                                translated  
1                        musieum sribaduga  
3             museum di dalam kota bandung  
4                                           
7  kesini pas study tour smp mungkin tahun  
8                             bagus sekali  


In [12]:
# 4. TOKENIZING

nltk.download('punkt')

def tokenizing(text):
    return nltk.word_tokenize(text)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
df['tokenized'] = df['translated'].apply(tokenizing)

In [14]:
# Kamus normalisasi (bisa ditambah)
normalization_dict = {
    # --- Kata Informal / Slang ---
    "ga": "tidak",
    "gk": "tidak",
    "gak": "tidak",
    "tdk": "tidak",
    "engga": "tidak",
    "enggak": "tidak",
    "nggak": "tidak",
    "ngga": "tidak",

    "bgt": "banget",
    "bngt": "banget",
    "bangettt": "banget",

    "cmn": "cuma",
    "cman": "cuma",
    "cuma2": "cuma",

    "trs": "terus",
    "trus": "terus",

    "tmn": "teman",
    "tmn2": "teman",

    "udh": "sudah",
    "udah": "sudah",
    "udhh": "sudah",

    "blm": "belum",
    "belom": "belum",

    "syg": "sayang",

    "krg": "kurang",

    "sblm": "sebelum",

    "dr": "dari",
    "drpd": "daripada",

    "klo": "kalau",
    "kalo": "kalau",

    "pls": "tolong",
    "plis": "tolong",

    "aja": "saja",
    "aj": "saja",

    "ni": "ini",
    "nih": "ini",

    "tu": "itu",
    "ituu": "itu",

    "bru": "baru",
    "baru2": "baru",

    "lok": "lokasi",

    "pas": "saat",

    # --- Singkatan Lokasi / Umum ---
    "bndng": "bandung",
    "bdg": "bandung",
    "jbr": "jawa barat",

    # --- Typos Umum ---
    "bgs": "bagus",
    "baguss": "bagus",
    "bgus": "bagus",

    "bnyk": "banyak",
    "bnyknya": "banyaknya",

    "tmpt": "tempat",
    "tempet": "tempat",

    "rame": "ramai",
    "ramee": "ramai",

    "nyamn": "nyaman",
    "nyamann": "nyaman",

    "lumayanlah": "lumayan",

    "mantappp": "mantap",
    "mantab": "mantap",
    "mantul": "mantap",

    # --- Istilah Museum / Edukasi ---
    "edukasii": "edukasi",
    "edukatif": "edukatif",
    "koleksinyaa": "koleksinya",
    "sejarahh": "sejarah",

    # --- Bahasa Inggris (umum di Google Maps) ---
    "good": "bagus",
    "nice": "bagus",
    "great": "bagus",
    "awesome": "bagus",
    "amazing": "bagus",
    "cool": "keren",
    "clean": "bersih",
    "recommended": "direkomendasikan",
    "recommend": "rekomendasi",
    "worth": "layak",
    "worthit": "layak",
    "worthwhile": "layak",
    "cheap": "murah",
    "expensive": "mahal",
    "great": "bagus",
    "nice": "bagus",

    # --- Bahasa Sunda (umum dalam ulasan Museum Sri Baduga) ---
    "teh": "",         # partikel (dapat dihapus)
    "mah": "",         # partikel penegas
    "atuh": "",        # ekspresi
    "pisan": "sekali",
    "teteh": "kakak",
    "kenging": "boleh",
    "alus": "bagus",
    "gawe": "kerja",

    # --- Variasi ejaan tidak baku ---
    "seblm": "sebelum",
    "seblumnya": "sebelumnya",
    "baguss": "bagus",
    "bersiih": "bersih",
    "bersiiih": "bersih",
    "bersihh": "bersih",

    "murahh": "murah",
    "murahhh": "murah",

    "lengkappp": "lengkap",

    "worthittt": "layak",

    # --- Kata sambung / filler ---
    "the": "the",          # tetap (kadang judul)
    "of": "of",            # tetap
    "ya": "",              # filler
    "yah": "",             # filler
    "loh": "",             # filler
    "kok": "",             # filler
    "kan": "",             # filler
    "lah": "",             # filler

    # --- Variasi Pengulangan Huruf (contoh umum) ---
    "bagusss": "bagus",
    "kerenn": "keren",
    "keren banget": "keren",
    "kenyanggg": "kenyang",
    "luasss": "luas",
    "seruu": "seru",
    "seruuu": "seru",
    "harganyaa": "harganya",

    # --- Kata Google Maps yang sering muncul ---
    "service": "layanan",
    "staff": "staf",
    "facility": "fasilitas",
    "facilities": "fasilitas",
    "parking": "parkir",
    "parkiran": "parkir",
    "recommended": "direkomendasikan",
    "experience": "pengalaman",
    "view": "pemandangan",
    "history": "sejarah",
    "historical": "sejarah",

    # --- Tambahan umum ---
    "yg": "yang",
    "aja": "saja",
    "gini": "begini",
    "gitu": "begitu",
    "sini": "sini",
    "situ": "situ",
    "kemaren": "kemarin",
    "kmrn": "kemarin",
    "kmrn2": "kemarin",
    "besok2": "besok",
    "jd": "jadi",
    "jg": "juga",
    "tll": "terlalu",
    "sgt": "sangat"
}

def normalize_text(tokens):
    return [normalization_dict.get(word, word) for word in tokens]


In [15]:
df['normalized'] = df['tokenized'].apply(normalize_text)

In [16]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

factory = StopWordRemoverFactory()
stopwords_id = set(factory.get_stop_words())

additional_stopwords = {
    "banget", "sekali", "kali",
    "ya", "yah", "iya",
    "sih", "nih", "tuh",
    "kayak", "kayaknya",
    "dong", "deh", "kok",
    "gitu", "gini",
    "aja", "doang",
    "bikin", "biar", "buat", "jadi",
    "tempat", "lokasi", "area",
    "museum", "to", "we", "about"
}

stopwords_final = stopwords_id.union(additional_stopwords)

def remove_stopwords(tokens):
    return [word for word in tokens if word not in stopwords_final]



In [17]:
df['stopword_removed'] = df['normalized'].apply(remove_stopwords)

In [18]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stem_sentence(tokens):
    sentence = " ".join(tokens)         # gabungkan list → string
    stemmed = stemmer.stem(sentence)    # stemming kalimat
    return stemmed.split()              # pecah lagi → list




In [19]:
df["stemmed_text"] = df["stopword_removed"].apply(stem_sentence)
df['stemmed_joined'] = df['stemmed_text'].apply(lambda x: " ".join(x))

In [20]:
import pandas as pd

# URL resmi InSet Lexicon
url_positive = "https://raw.githubusercontent.com/fajri91/InSet/master/positive.tsv"
url_negative = "https://raw.githubusercontent.com/fajri91/InSet/master/negative.tsv"

# Membaca lexicon dari GitHub tanpa header bawaan file
lex_pos = pd.read_csv(url_positive, sep="\t", header=None, names=["word", "weight"], skiprows=1)
lex_neg = pd.read_csv(url_negative, sep="\t", header=None, names=["word", "weight"], skiprows=1)

# Gabungkan
lexicon_df = pd.concat([lex_pos, lex_neg], ignore_index=True)

# Konversi weight ke integer (AMAN sekarang)
lexicon_df["weight"] = lexicon_df["weight"].astype(int)

# Jadikan dictionary
lexicon = dict(zip(lexicon_df["word"], lexicon_df["weight"]))

print("Jumlah kata dalam lexicon:", len(lexicon))


Jumlah kata dalam lexicon: 9074


In [21]:
# Ambil bobot maksimum untuk tiap kata → lebih logis untuk sentimen
lexicon_clean = lexicon_df.groupby("word", as_index=False)["weight"].max()

# Ubah ke dictionary
lexicon_final = dict(zip(lexicon_clean["word"], lexicon_clean["weight"]))

print("Jumlah kata di lexicon_final:", len(lexicon_final))


Jumlah kata di lexicon_final: 9074


In [22]:
def get_lexicon_score(tokens, lexicon):
    if not isinstance(tokens, list):
        return 0
    return sum(lexicon.get(word, 0) for word in tokens)

df["lexicon_score"] = df["stemmed_text"].apply(lambda x: get_lexicon_score(x, lexicon_final))


In [23]:
def label_sentiment(score):
    if score > 0:
        return "positif"
    elif score < 0:
        return "negatif"
    else:
        return "netral"

df["sentiment"] = df["lexicon_score"].apply(label_sentiment)


In [24]:
df[["stemmed_text", "lexicon_score", "sentiment"]].head(20)


,stemmed_text,lexicon_score,sentiment
1,"[musieum, sribaduga]",0,netral
3,"[kota, bandung]",-2,negatif
4,[],0,netral
7,"[kesini, study, tour, smp, mungkin, tahun]",-1,negatif
8,[bagus],2,positif
9,[mantap],5,positif
10,"[seru, anak, tau, sejarah, banyak, ajar]",2,positif
16,[mantap],5,positif
17,"[sangat, baik, edukasi, anak, didik, koleksi, ...",9,positif
18,"[datang, sini, waktu, pamer, nasional, pusaka,...",3,positif


In [25]:
df['sentiment'].value_counts().reset_index().rename(columns={'index': 'sentiment', 'sentiment': 'jumlah'})


,jumlah,count
0,positif,1044
1,negatif,228
2,netral,220


In [26]:
tfidf = TfidfVectorizer(
    ngram_range=(1,2),   # unigram + bigram
    min_df=2,            # kata minimal muncul 2 dokumen
    max_df=0.95          # membuang kata terlalu umum
)


In [27]:
X_tfidf = tfidf.fit_transform(df["stemmed_joined"])


In [28]:
df["sentiment"]   # hasil dari lexicon (positif/negatif/netral)


,sentiment
1,netral
3,negatif
4,netral
7,negatif
8,positif
...,...
3644,positif
3645,positif
3648,positif
3655,negatif


In [29]:
y = df["sentiment"]


In [30]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)


In [35]:
feature_names = tfidf.get_feature_names_out()
feature_names[:50]   # tampilkan 50 kata pertama


array(['abad', 'abadi', 'abadi buah', 'ac', 'acara', 'ad', 'ada',
       'ada depan', 'ada jawa', 'ada tengah', 'adab', 'adab jawa',
       'adab manusia', 'adab masa', 'adat', 'adem', 'aga', 'agam',
       'agama', 'air', 'air kecil', 'ajak', 'ajak anak', 'ajak main',
       'ajar', 'ajar ambil', 'ajar anak', 'ajar asal', 'ajar bagai',
       'ajar bandung', 'ajar banyak', 'ajar budaya', 'ajar ilmu',
       'ajar jaman', 'ajar kenal', 'ajar kisah', 'ajar lebih',
       'ajar mahasiswa', 'ajar main', 'ajar sangat', 'ajar sayang',
       'ajar sejarah', 'ajar tiket', 'ajar ttg', 'ajar wajib', 'akhir',
       'akhir kesini', 'akhir tahun', 'aksara', 'akses'], dtype=object)

In [56]:
doc_index = 0  # pilih dokumen ke-1
df_tfidf_doc = pd.DataFrame(
    X_tfidf[doc_index].T.toarray(),
    index=feature_names,
    columns=["TF-IDF"]
)

# Tampilkan kata dengan bobot terbesar
df_tfidf_doc.sort_values("TF-IDF", ascending=False).head(20)


,TF-IDF
musieum sribaduga,0.641758
musieum,0.596289
sribaduga,0.482273
parkir nyaman,0.000000
pasar,0.000000
pasundan,0.000000
patung,0.000000
patung rem,0.000000
pamer pusaka,0.000000
pdhl,0.000000


In [55]:
#Cek beberapa fitur dengan nilai bobot
#Menampilkan bobot kata pada dokumen tertentu & Memastikan ada nilai TF-IDF yang bervariasi (tidak semuanya 0)
import numpy as np

tfidf_mean = np.mean(X_tfidf.toarray(), axis=0)
top_indices = np.argsort(tfidf_mean)[-20:]

for i in top_indices:
    print(feature_names[i], tfidf_mean[i])


koleksi 0.014318381611384243
sunda 0.015179610991810909
nya 0.015227499671486466
jawa barat 0.016395443607001167
budaya 0.016425511245468507
bersih 0.016528745831020247
barat 0.01666632678827673
jawa 0.016707994298339163
tahu 0.017470681447086706
tempat 0.01815356522562596
bandung 0.018489123500604353
sangat 0.01922248755148281
mantap 0.019693738435122333
edukasi 0.019703644914149356
nyaman 0.019763594860585647
ajar 0.021265138281937268
banyak 0.02269271042876924
anak 0.025111520712105432
sejarah 0.05368119648165565
bagus 0.08430075467314543


In [57]:
df_tfidf = pd.DataFrame(X_tfidf.toarray(), columns=feature_names)
df_tfidf.head()


,abad,abadi,abadi buah,ac,acara,ad,ada,ada depan,ada jawa,ada tengah,...,yo,you,yuk,yusuf,zaman,zaman batu,zaman dulu,zaman kala,zaman purba,zaman purbakala
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
#Artinya TF-IDF berhasil menghasilkan 5000 fitur untuk 1492 dokumen.
print("TF-IDF shape:", X_tfidf.shape)


TF-IDF shape: (1492, 2062)


In [42]:
#Matriks TF-IDF bersifat sparse (banyak 0), tapi harus memiliki beberapa nilai bukan nol.
#Jika hasilnya > 0, maka pembobotan berhasil.
X_tfidf.nnz


14712

In [64]:
# Interpretasi density kamu (0.00478)
# Matriks TF-IDF sangat sparse (bagus)
# Preprocessing berhasil (karena ada pola kata tetapi tidak terlalu padat)
# TF-IDF berfungsi normal
# SVM Linear akan bekerja optimal
# Nilai ini persis seperti matriks TF-IDF pada penelitian sentimen real (biasanya density 0.1% – 1%).

density = X_tfidf.nnz / (X_tfidf.shape[0] * X_tfidf.shape[1])
density


0.004782051315389156

In [65]:
#mengecek missing value
X_tfidf = tfidf.fit_transform(df['stemmed_joined'])


In [66]:
df['stemmed_joined'].isnull().sum()


np.int64(0)

In [67]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)


In [68]:
print("Data latih (X_train):", X_train.shape[0])
print("Data uji (X_test):", X_test.shape[0])
print("Label latih (y_train):", len(y_train))
print("Label uji (y_test):", len(y_test))


Data latih (X_train): 1193
Data uji (X_test): 299
Label latih (y_train): 1193
Label uji (y_test): 299
